<a href="https://colab.research.google.com/github/TahaMsv/Bachelor-Project/blob/main/bachelor_project(Other_models).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import os
import mne
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# base_path = "/drive/Shared with me/Bachelor's project/UC San Diego/"
base_path = "/content/drive/MyDrive/Bachelor's project/UC San Diego/"

#Load data

In [ ]:
data =np.load(base_path + "pre processed data.npy")
labels = np.load(base_path + "pre processed labels.npy")

In [ ]:
print(data.shape)
print(labels.shape)

(2880, 41, 1024)
(2880,)


#LSTM

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras
# Define the number of classes and the number of folds for cross-validation
num_classes = 2  # 2 classes for binary classification (Parkinson's and non-Parkinson's)
num_folds = 10

# Initialize K-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
lstm_accuracies = []
lstm_sensitivities = []
lstm_specificities = []

# Define LSTM model
def create_lstm_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(128, input_shape=(41, 1024), return_sequences=True))
    model.add(keras.layers.LSTM(64, return_sequences=False))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer= keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
    return model

# Perform 10-fold cross-validation
# Perform 10-fold cross-validation
for fold, (train_index, test_index) in enumerate(skf.split(data, labels), 1):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Create and train the LSTM model
    model = create_lstm_model()
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Make predictions
    y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)

    lstm_accuracies.append(accuracy)
    lstm_sensitivities.append(sensitivity)
    lstm_specificities.append(specificity)
    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")


# Calculate the mean and standard deviation of evaluation metrics
mean_accuracy = np.mean(lstm_accuracies)
mean_sensitivity = np.mean(lstm_sensitivities)
mean_specificity = np.mean(lstm_specificities)

std_accuracy = np.std(lstm_accuracies)
std_sensitivity = np.std(lstm_sensitivities)
std_specificity = np.std(lstm_specificities)

print("Mean Accuracy:", mean_accuracy)
print("Mean Sensitivity (True Positive Rate):", mean_sensitivity)
print("Mean Specificity (True Negative Rate):", mean_specificity)
print("Standard Deviation of Accuracy:", std_accuracy)
print("Standard Deviation of Sensitivity:", std_sensitivity)
print("Standard Deviation of Specificity:", std_specificity)


9/9 [==============================] - 2s 69ms/step
Fold 1 - Accuracy: 0.9549, Sensitivity: 1.0000, Specificity: 0.9097
9/9 [==============================] - 1s 39ms/step
Fold 2 - Accuracy: 0.8681, Sensitivity: 1.0000, Specificity: 0.7361
9/9 [==============================] - 1s 40ms/step
Fold 3 - Accuracy: 0.9583, Sensitivity: 1.0000, Specificity: 0.9167
9/9 [==============================] - 1s 54ms/step
Fold 4 - Accuracy: 0.9861, Sensitivity: 1.0000, Specificity: 0.9722
9/9 [==============================] - 1s 39ms/step
Fold 5 - Accuracy: 0.9583, Sensitivity: 1.0000, Specificity: 0.9167
9/9 [==============================] - 1s 47ms/step
Fold 6 - Accuracy: 0.9965, Sensitivity: 0.9931, Specificity: 1.0000
9/9 [==============================] - 1s 43ms/step
Fold 7 - Accuracy: 0.9444, Sensitivity: 1.0000, Specificity: 0.8889
9/9 [==============================] - 1s 39ms/step
Fold 8 - Accuracy: 0.9688, Sensitivity: 0.9931, Specificity: 0.9444
9/9 [==============================] - 1

#SVM

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix


# Define the number of classes and the number of folds for cross-validation
num_classes = 2  # 2 classes for binary classification (Parkinson's and non-Parkinson's)
num_folds = 10

# Initialize K-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
svm_accuracies = []
svm_sensitivities = []
svm_specificities = []

# Perform 10-fold cross-validation
for fold, (train_index, test_index) in enumerate(skf.split(data, labels), 1):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Create and train the SVM classifier
    clf = svm.SVC(kernel='sigmoid')  # You can try different kernels

    clf.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Flatten the input data

    # Make predictions
    y_pred = clf.predict(X_test.reshape(X_test.shape[0], -1))  # Flatten the test data

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)

    svm_accuracies.append(accuracy)
    svm_sensitivities.append(sensitivity)
    svm_specificities.append(specificity)
    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")


# Calculate the mean and standard deviation of evaluation metrics
mean_accuracy = np.mean(svm_accuracies)
mean_sensitivity = np.mean(svm_sensitivities)
mean_specificity = np.mean(svm_specificities)

std_accuracy = np.std(svm_accuracies)
std_sensitivity = np.std(svm_sensitivities)
std_specificity = np.std(svm_specificities)

print("Mean Accuracy:", mean_accuracy)
print("Mean Sensitivity (True Positive Rate):", mean_sensitivity)
print("Mean Specificity (True Negative Rate):", mean_specificity)
print("Standard Deviation of Accuracy:", std_accuracy)
print("Standard Deviation of Sensitivity:", std_sensitivity)
print("Standard Deviation of Specificity:", std_specificity)


Fold 1 - Accuracy: 0.4965, Sensitivity: 0.0069, Specificity: 0.9861
Fold 2 - Accuracy: 0.4931, Sensitivity: 0.0000, Specificity: 0.9861
Fold 3 - Accuracy: 0.4931, Sensitivity: 0.0000, Specificity: 0.9861
Fold 4 - Accuracy: 0.5000, Sensitivity: 0.0069, Specificity: 0.9931
Fold 5 - Accuracy: 0.4965, Sensitivity: 0.9931, Specificity: 0.0000
Fold 6 - Accuracy: 0.5035, Sensitivity: 0.0069, Specificity: 1.0000
Fold 7 - Accuracy: 0.5035, Sensitivity: 0.0139, Specificity: 0.9931
Fold 8 - Accuracy: 0.5000, Sensitivity: 0.0069, Specificity: 0.9931
Fold 9 - Accuracy: 0.5000, Sensitivity: 0.0139, Specificity: 0.9861
Fold 10 - Accuracy: 0.4931, Sensitivity: 0.9861, Specificity: 0.0000
Mean Accuracy: 0.4979166666666666
Mean Sensitivity (True Positive Rate): 0.20347222222222222
Mean Specificity (True Negative Rate): 0.7923611111111111
Standard Deviation of Accuracy: 0.0038665030297430573
Standard Deviation of Sensitivity: 0.3930831605932456
Standard Deviation of Specificity: 0.3962041391851017


#Save results

In [ ]:
#LSTM results
lstm_results = {
        'accuracies': lstm_accuracies,
        'sensitivities': lstm_sensitivities,
        'specificities': lstm_specificities
    }

svm_results = {
        'accuracies': svm_accuracies,
        'sensitivities': svm_sensitivities,
        'specificities': svm_specificities
    }

np.save(base_path + "lstm_results.npy", lstm_results)
np.save(base_path + "svm_results.npy", svm_results)

#Load results

In [ ]:
loaded_lstm_results = np.load(base_path + "lstm_results.npy", allow_pickle=True).item()
loaded_svm_results = np.load(base_path + "svm_results.npy", allow_pickle=True).item()

# Access the metrics for specific models (e.g., LSTM and SVM)
lstm_accuracies = loaded_lstm_results['accuracies']
lstm_sensitivities = loaded_lstm_results['sensitivities']
lstm_specificities = loaded_lstm_results['specificities']

svm_accuracies = loaded_svm_results['accuracies']
svm_sensitivities = loaded_svm_results['sensitivities']
svm_specificities = loaded_svm_results['specificities']